### Display RDD
The following helper functions displays the current contents of an RDD (partition-by-partition). This is best used for small RDDs with manageable number of partitions.

In [1]:
class DisplayRDD:
        def __init__(self, rdd):
                self.rdd = rdd

        def _repr_html_(self):                                  
                x = self.rdd.mapPartitionsWithIndex(lambda i, x: [(i, [y for y in x])])
                l = x.collect()
                s = "<table><tr>{}</tr><tr><td>".format("".join(["<th>Partition {}".format(str(j)) for (j, r) in l]))
                s += '</td><td valign="bottom" halignt="left">'.join(["<ul><li>{}</ul>".format("<li>".join([str(rr) for rr in r])) for (j, r) in l])
                s += "</td></table>"
                return s

### Initialize the RDDs for Assignment

In [2]:
playRDD = sc.textFile("datafiles/play.txt")
logsRDD = sc.textFile("datafiles/NASA_logs_sample.txt")
amazonInputRDD = sc.textFile("datafiles/amazon-ratings.txt")
nobelRDD = sc.textFile("datafiles/prize.json")

In [3]:
amazonBipartiteRDD = amazonInputRDD.map(lambda x: x.split(" ")).map(lambda x: (x[0], x[1])).distinct()

In [4]:
print(amazonBipartiteRDD.take(10))

[('user1', 'product1'), ('user1', 'product2'), ('user1', 'product3'), ('user1', 'product4'), ('user1', 'product5'), ('user1', 'product6'), ('user1', 'product7'), ('user1', 'product8'), ('user1', 'product9'), ('user1', 'product10')]


In [5]:
print(logsRDD.take(10))

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245', 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985', '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085', 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0', '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179', 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0', 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0', '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985', 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985', '129.94.144.152 - - [01/Jul/1995:00:0

In [4]:
def f(line):
    sp = line.split()
    return (sp[0][4:], sp[1][7:], sp[2])

In [5]:
x = amazonInputRDD.map(f)

In [6]:
print(x.takeOrdered(10))

[('1', '1', '5.0'), ('1', '10', '1.0'), ('1', '11', '4.0'), ('1', '12', '5.0'), ('1', '13', '5.0'), ('1', '14', '5.0'), ('1', '15', '4.0'), ('1', '16', '5.0'), ('1', '17', '5.0'), ('1', '18', '2.0')]


In [15]:
def f2(line):
    sp = line.split()
    return (sp[0], float(sp[2]))

In [18]:
x2 = amazonInputRDD.map(f2).aggregateByKey((0, 0), lambda x, y: (x[0] + y, x[1] + 1), lambda x, y: (x[0] + y[0], x[1] + y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))

In [19]:
print(x2.takeOrdered(10))

[('user1', 4.08), ('user10', 3.227272727272727), ('user100', 4.666666666666667), ('user101', 4.928571428571429), ('user102', 4.0), ('user103', 4.3), ('user104', 3.7142857142857144), ('user105', 3.75), ('user106', 3.679144385026738), ('user107', 4.071428571428571)]


In [22]:
def f3(line):
    sp = line.split()
    return (sp[1], sp[2])
def computemode(arr):
    arr = list(arr)
    maxcount = max([arr.count(x) for x in set(arr)])
    for x in ['5.0', '4.0', '3.0', '2.0', '1.0']:
        if arr.count(x) == maxcount:
            return x
x3 = amazonInputRDD.map(f3).groupByKey().mapValues(computemode)

In [23]:
print(x3.takeOrdered(10))

[('product0', '5.0'), ('product1', '5.0'), ('product10', '5.0'), ('product100', '5.0'), ('product101', '5.0'), ('product102', '5.0'), ('product103', '5.0'), ('product104', '5.0'), ('product105', '5.0'), ('product106', '5.0')]


In [28]:
def extractYear(logline):
        match = re.search('/([0-9][0-9][0-9][0-9]):', logline)
        return (match.group(1), 1)
x4  = logsRDD.map(extractYear).reduceByKey(lambda x, y: x+y)

In [27]:
import re

In [29]:
print(x4.takeOrdered(10))

[('1995', 10000)]
